In [26]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')

In [27]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(QQ)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)

In [28]:
L_set_base = ('E1', 'G4', 'E2', 'G3', 'E3')

In [29]:
possible_L_sets = [("E1", "G4", "E2", "F12", "E3"), \
("E1", "G4", "E2", "G5", "F34"), ("E1", "G4", "E2", "G6", "F34"), \
("E1", "G4", "F24", "G2", "F34"), ("E1", "G4", "F24", "F13", "F34"), \
("F14", "G4", "F24", "F35", "F34"), ("F14", "G4", "F24", "F36", "F34"), \
("E1", "G4", "E3", "G2", "E2"), ("E1", "G4", "E3", "F13", "E2"), \
("F14", "G4", "E3", "F35", "E2"), ("F14", "G4", "E3", "F36", "E2")]                 

In [30]:
%time L_set_ideals, all_ideals = general_cubic.find_conditions_on_L_sets(possible_L_sets)

CPU times: user 8.49 s, sys: 23.7 ms, total: 8.51 s
Wall time: 8.55 s


In [31]:
cubics = [general_cubic.reduce(ideal) for ideal in all_ideals]
[len(cubic.eckardt_points_labels) for cubic in cubics]

[2, 2, 2, 2, 1, 1, 1, 2, 2, 6, 6]